In [36]:
if __name__ == '__main__':
    import os
    from pprint import pprint

    from msl.equipment import EquipmentRecord, ConnectionRecord, Backend

    from msl.equipment.resources.thorlabs import MotionControl
    # ensure that the Kinesis folder is available on PATH
    os.environ['PATH'] += os.pathsep + 'C:/Program Files/Thorlabs/Kinesis'

    print('Building the device list...')
    MotionControl.build_device_list()
    
    n_devices = MotionControl.get_device_list_size()
    if n_devices == 0:
        print('There are no devices in the device list')
        sys.exit(0)
    elif n_devices == 1:
        print('There is 1 device in the device list')
    else:
        print('There are {} devices in the device list'.format(n_devices))

    all_devices = MotionControl.get_device_list()
    print('The serial numbers of all the devices are: {}'.format(all_devices))
    
    # rather than reading the EquipmentRecord from a database we can create it manually
    record = EquipmentRecord(
        manufacturer='Thorlabs',
        model='KST101',
        serial='26001683',  # update the serial number for your KST101
        connection=ConnectionRecord(
            backend=Backend.MSL,
            address='SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll',
        ),
    )
    record2 = EquipmentRecord(
    manufacturer='Thorlabs',
    model='KST101',
    serial='26001749',  # update the serial number for your KST101
    connection=ConnectionRecord(
        backend=Backend.MSL,
        address='SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll',
    ),
)

Building the device list...
There are 2 devices in the device list
The serial numbers of all the devices are: ['26001683', '26001749']


In [37]:
def wait(value,motor):
        motor.clear_message_queue()
        message_type, message_id, _ = motor.wait_for_message()
        while message_type != 2 or message_id != value:
            position = motor.get_position()
            real = motor.get_real_value_from_device_unit(position, 'DISTANCE')
            print('  at position {} [device units] {} [real-world units]'.format(position, real))
            message_type, message_id, _ = motor.wait_for_message()

In [38]:
motory = record2.connect()
print('Connected to {}'.format(motory))
# connect to the KCube Stepper Motor
motorx = record.connect()
print('Connected to {}'.format(motorx))

Connected to KCubeStepperMotor<Thorlabs|KST101|26001749>
Connected to KCubeStepperMotor<Thorlabs|KST101|26001683>


In [39]:
  # load the configuration settings (so that we can use the get_real_value_from_device_unit() method)
motory.load_settings()
motorx.load_settings()

In [40]:
 # start polling at 200 ms
motorx.start_polling(200)
motory.start_polling(200)

In [41]:
# home the device
print('Homing...')
motorx.home()
wait(0,motorx)
print('Homing done. At position {} [device units]'.format(motorx.get_position()))

Homing...
  at position -13 [device units] -6.5e-06 [real-world units]
  at position -13 [device units] -6.5e-06 [real-world units]
  at position -75381 [device units] -0.0375287 [real-world units]
  at position -267706 [device units] -0.1332783 [real-world units]
  at position -283142 [device units] -0.1409632 [real-world units]
  at position -283142 [device units] -0.1409632 [real-world units]
  at position -242737 [device units] -0.1208474 [real-world units]
  at position -202348 [device units] -0.1007396 [real-world units]
  at position -202348 [device units] -0.1007396 [real-world units]
  at position 19466 [device units] 0.0096912 [real-world units]
  at position 59853 [device units] 0.029798 [real-world units]
  at position 100246 [device units] 0.0499078 [real-world units]
  at position 140664 [device units] 0.07003 [real-world units]
  at position 181087 [device units] 0.0901548 [real-world units]
  at position 0 [device units] 0.0 [real-world units]
  at position 0 [device un

In [42]:
print('Homing...')
motory.home()
wait(0,motory)
print('Homing done. At position {} [device units]'.format(motory.get_position()))

Homing...
  at position 399964 [device units] 0.1991234 [real-world units]
  at position 399964 [device units] 0.1991234 [real-world units]
  at position 302239 [device units] 0.1504707 [real-world units]
  at position 104096 [device units] 0.0518245 [real-world units]
  at position -96853 [device units] -0.0482186 [real-world units]
  at position -297741 [device units] -0.1482314 [real-world units]
  at position -344338 [device units] -0.1714298 [real-world units]
  at position -344338 [device units] -0.1714298 [real-world units]
  at position -303941 [device units] -0.151318 [real-world units]
  at position -263527 [device units] -0.1311978 [real-world units]
  at position -223127 [device units] -0.1110845 [real-world units]
  at position 6514 [device units] 0.003243 [real-world units]
  at position 6514 [device units] 0.003243 [real-world units]
  at position 43123 [device units] 0.0214689 [real-world units]
  at position 83522 [device units] 0.0415817 [real-world units]
  at positi

In [49]:
def move_to_x_y_position(x,y):
    motorx.move_to_position(motorx.get_device_unit_from_real_value(x,0))
    motory.move_to_position(motory.get_device_unit_from_real_value(y,0))
    print('Moving done. At position {} [device units]'.format(motorx.get_position()))
    print('Moving done. At position {} [device units]'.format(motory.get_position()))

In [50]:
move_to_x_y_position(1.0,1.0)

Moving done. At position 20086236 [device units]
Moving done. At position 20086236 [device units]


In [51]:
motory.disconnect()
motorx.disconnect()